In [1]:
# raw APL with background, cell containers to be inserted
apl = """
{
  "type": "APL",
  "version": "1.3",
  "settings": {},
  "theme": "dark",
  "import": [
    {
      "name": "alexa-layouts",
      "version": "1.5.0"
    }
  ],
  "resources": [],
  "styles": {
    "cellContainerStyle": {
      "values": [
        {
          "opacity": "0.8"
        }
      ]
    },
    "cellStyle": {
      "values": [
        {
          "fontSize": "25dp",
          "textAlign": "center",
          "textAlignVertical": "center"
        }
      ]
    }
  },
  "onMount": [],
  "graphics": {},
  "commands": {},
  "layouts": {},
  "mainTemplate": {
    "parameters": [
      "payload"
    ],
    "items": [
      {
        "type": "Container",
        "width": "100%",
        "height": "100%",
        "direction": "column",
        "items": [
          {
            "type": "AlexaBackground",
            "backgroundImageSource": "https://www.bildung-forschung.digital/digitalezukunft/de/bildung/mint-allianz/was-wimmelt-in-mint/_functions/wimmelbild_png.png?__blob=normal&amp;v=2&amp;.png",
            "colorOverlay": true,
            "backgroundScale": "bestfill"
          },
          CELLCONTAINERS
            ]
          }
        ]
      }
}
"""

In [4]:
def generate(n_rows, n_cols, apl=apl):
    # row container draft
    s_row = """
            {
            "type": "Container",
            "id": "row-RRR",
            "width": "100%",
            "height": "20%",
            "direction": "row",
            "items": [
            """
    # cell container draft
    s_cell = """
                {
                "type": "Container",
                "id": "row-RRR-column-CCC",
                "width": "20%",
                "height": "100%",
                "style": "cellContainerStyle",
                "items": [
                  {
                    "type": "TouchWrapper",
                    "item": {
                      "type": "Text",
                      "id": "cell-RRR-CCC",
                      "width": "100%",
                      "height": "100%",
                      "style": "cellStyle",
                      "text": "cell-RRR-CCC",
                      "color": "#66DFFF",
                      "fontSize": 25
                    },
                    "onPress": [
                      {
                        "type": "SendEvent",
                        "arguments": [
                          "items_dict[]"
                        ]
                      }
                    ]
                  }
                  ]
                  }
                  """
    cellcontainers = """"""
    for r in range(n_rows):
        # initialize row container
        sr = s_row.replace('RRR', str(r))
        temp = sr
        for c in range(n_cols):
            # add cell containers
            sc = s_cell.replace('RRR', str(r))
            sc = sc.replace('CCC', str(c))
            if c == (n_cols-1):  # no comma after last cell in a row, close items list
                if r == (n_rows-1):  # no comma after last row, close items list
                    temp = temp + sc + ']}'
                else:
                    temp = temp + sc + ']},'
            else:  # comma and next cell
                temp = temp + sc + ',\n'
        print(temp)
        cellcontainers += temp
    # insert into APL file
    output = apl.replace('CELLCONTAINERS', cellcontainers)
    with open('cellcontainer.txt', 'w', encoding='utf-8') as f:
        f.write(output)

In [5]:
generate(6,8)


            {
            "type": "Container",
            "id": "row-0",
            "width": "100%",
            "height": "20%",
            "direction": "row",
            "items": [
            
                {
                "type": "Container",
                "id": "row-0-column-0",
                "width": "20%",
                "height": "100%",
                "style": "cellContainerStyle",
                "items": [
                  {
                    "type": "TouchWrapper",
                    "item": {
                      "type": "Text",
                      "id": "cell-0-0",
                      "width": "100%",
                      "height": "100%",
                      "style": "cellStyle",
                      "text": "cell-0-0",
                      "color": "#66DFFF",
                      "fontSize": 25
                    },
                    "onPress": [
                      {
                        "type": "SendEvent",
                       